In [4]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

In [5]:
data = pd.read_csv('../data/data_msg_only(new tags).csv')

In [6]:
def custom_round(x, base=10):
    return '_' + str(int(base * round(float(x)/base)))

In [7]:
def map_and_or_count(x):
    if x == 0:
        return '0_and_or_count'
    if x == 1:
        return '1_and_or_count'

In [8]:
def map_blank_line(x):
    if x == 0:
        return '0_blank_line'
    if x == 1:
        return '1_blank_line'

In [9]:
def map_capital_start(x):
    if x == 0:
        return '0_capital_start'
    if x == 1:
        return '1_capital_start'

In [10]:
def map_end_dot(x):
    if x == 0:
        return '0_end_dot'
    if x == 1:
        return '1_end_dot'

In [11]:
def map_imperative_start(x):
    if x == 0:
        return '0_imperative_start'
    if x == 1:
        return '1_imperative_start'

In [12]:
def map_wrap_to_72(x):
    if x == 0:
        return '0_wrap_to_72'
    if x == 1:
        return '1_wrap_to_72'

In [13]:
def map_verb_direct_obj(x):
    if x == 0:
        return '0_wrap_to_72'
    if x == 1:
        return '1_wrap_to_72'

In [14]:
data['characters_count'] = pd.Series(data['characters_count']).apply(lambda x: custom_round(x, base=5))
data['and_or_count'] = pd.Series(data['and_or_count']).apply(lambda x: map_and_or_count(x))
data['blank_line'] = pd.Series(data['blank_line']).apply(lambda x: map_blank_line(x))
data['capital_start'] = pd.Series(data['capital_start']).apply(lambda x: map_capital_start(x))
data['end_dot'] = pd.Series(data['end_dot']).apply(lambda x: map_end_dot(x))
data['imperative_start'] = pd.Series(data['imperative_start']).apply(lambda x: map_imperative_start(x))
data['wrap_to_72'] = pd.Series(data['wrap_to_72']).apply(lambda x: map_wrap_to_72(x))
data['verb_direct_obj'] = pd.Series(data['verb_direct_obj']).apply(lambda x: map_verb_direct_obj(x))

In [15]:
data.head()

,characters_count,and_or_count,blank_line,capital_start,end_dot,imperative_start,wrap_to_72,verb_direct_obj,label
0,_30,1_and_or_count,1_blank_line,1_capital_start,1_end_dot,1_imperative_start,1_wrap_to_72,1_wrap_to_72,good
1,_35,1_and_or_count,1_blank_line,1_capital_start,1_end_dot,1_imperative_start,1_wrap_to_72,1_wrap_to_72,good
2,_45,1_and_or_count,1_blank_line,1_capital_start,0_end_dot,1_imperative_start,1_wrap_to_72,1_wrap_to_72,good
3,_40,1_and_or_count,1_blank_line,1_capital_start,1_end_dot,1_imperative_start,1_wrap_to_72,1_wrap_to_72,good
4,_30,1_and_or_count,1_blank_line,1_capital_start,1_end_dot,1_imperative_start,1_wrap_to_72,1_wrap_to_72,good


In [16]:
dataset  = data.values.tolist()

In [17]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [18]:
frequent_itemsets = fpgrowth(df, min_support=0.2, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.975,(1_blank_line)
1,0.972,(1_wrap_to_72)
2,0.880,(1_and_or_count)
3,0.791,(1_end_dot)
4,0.689,(1_capital_start)
...,...,...
207,0.275,"(1_end_dot, 1_blank_line, 0_imperative_start, ..."
208,0.268,"(1_end_dot, 1_wrap_to_72, 0_imperative_start, ..."
209,0.221,"(0_wrap_to_72, 0_imperative_start, 1_wrap_to_72)"
210,0.215,"(0_wrap_to_72, 0_imperative_start, 1_blank_line)"


In [19]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

rules_good = rules[rules['consequents'].astype(str).str.contains('{\'good\'}')]
rules_neutral = rules[rules['consequents'].astype(str).str.contains('{\'neutral\'}')]
rules_bad = rules[rules['consequents'].astype(str).str.contains('{\'bad\'}')]

rules_good
# rules_neutral
# rules_bad
# rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
428,"(1_end_dot, 1_imperative_start, 1_wrap_to_72, ...",(good),0.395,0.413,0.278,0.703797,1.704110,0.114865,1.981752
452,"(1_end_dot, 1_wrap_to_72, 1_and_or_count, 1_im...",(good),0.395,0.413,0.278,0.703797,1.704110,0.114865,1.981752
688,"(1_end_dot, 1_capital_start, 1_imperative_star...",(good),0.347,0.413,0.244,0.703170,1.702591,0.100689,1.977563
700,"(1_end_dot, 1_imperative_start, 1_capital_star...",(good),0.309,0.413,0.233,0.754045,1.825776,0.105383,2.386618
712,"(1_end_dot, 1_capital_start, 1_wrap_to_72, 1_i...",(good),0.347,0.413,0.244,0.703170,1.702591,0.100689,1.977563
737,"(1_end_dot, 1_capital_start, 1_and_or_count, 1...",(good),0.308,0.413,0.233,0.756494,1.831703,0.105796,2.410613
760,"(1_end_dot, 1_capital_start, 1_wrap_to_72, 1_a...",(good),0.303,0.413,0.233,0.768977,1.861930,0.107861,2.540871
784,"(1_end_dot, 1_wrap_to_72, 1_capital_start, 1_a...",(good),0.303,0.413,0.233,0.768977,1.861930,0.107861,2.540871
